# Generate intraday bars using bars data service - F#

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot `intraday bars` with a **customizable** time range sampling.<br>

It enables to retrieve **on-demand** data by calling a dedicated bars service on a tick by tick data store. 

### Inputs/outputs
Bars sample requires instrument's identifier, date time intervals as per inputs. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve bars from the dedicated hosted service. The queried endpoint in this script are:
* *TickBarsService*: to directly retrieve bars objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.TickAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run bars sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.TickAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Create and process request
To request *tick bars service*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bars request parameters

#### 3.1 Instrument selection

In [ ]:
let ticker = "AAPL"
let exchange = "BATS"

#### 3.2 Bars calculation parameters

In [ ]:
// Set the bar duration
let sampling = 5L * 60L

// Set the bar calculation field
let field = BarPrice.Trade 

#### 3.3 Time period delimitation

In [ ]:
// Create time intervals
let yesterday = DateTime.Today.AddDays(-1.)
let start = yesterday.AddDays(-50.)

// Build the bars request date interval (we are using Google date time format : we have to cast the dates)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = start.Year, Month = start.Month, Day = start.Day ),
    EndDate = new Date ( Year = yesterday.Year, Month = yesterday.Month, Day = yesterday.Day )
  )

// Build the bars request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
let timeInterval = 
  new TimeInterval(
    StartTime = new TimeOfDay ( Hours = 15, Minutes = 00, Seconds = 00 ),
    EndTime = new TimeOfDay ( Hours = 16, Minutes = 00, Seconds = 00 ) 
  )

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous space selection:
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [ ]:
// Generate intraday bars request
let request = 
    new TickBarsRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker), 
        Constraints = constraints,  
        Sampling = new Duration (Seconds = sampling),
        Field = field
    )

In [ ]:
// Instantiate the intraday bars service
let service = new TickBarsService.TickBarsServiceClient(channel)

In [ ]:
// Process intraday bars request
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let bars = new List<TickBarsResponse>()

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()
    
let call = service.TickBars(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e bars.Add


In [ ]:
// Calculate the total ticks inside all the bars we received
let ticks = bars |> Seq.sumBy (fun bar -> bar.Count)

// Display
Console.WriteLine($"Fetched {bars.Count:N0} bars from (Total trades={ticks:N0})")
display(bars)

### Step 4: Visualize data

#### 4.1 Plot the bars Chart
Plot the bars request result and plot them with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
// Plot selected bars
bars 
|> Seq.map (fun b -> (b.TimeStamp.ToDateTime(), b.Open, b.High, b.Low, b.Close))
|> Chart.Candlestick
|> Chart.WithTitle $"{sampling/60L}-min Bars for {ticker}"
|> Chart.WithXTitle "Time"
|> Chart.WithYTitle "OHLC"
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display